In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "accredo_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::accredo_cancel_before_active
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    # Column headers
    status_date: str = 'status_date'# Status date column
    ref_date: str = 'rec_date'# Referral date column
    patient: str = 'msa_patient_id' # Patient ID column
    pharm: str = 'pharm_code' # Pharmacy Name column
    product: str = 'medication' # Medication Name column
    status: str = 'status_code' # Status column
    substatus: str = 'sub_status' # Substatus column
    ic_status: str = 'integrichain_status' # IntegriChain Status column
    ic_substatus: str = 'integrichain_sub_status' # IntegriChain Substatus column
    pjh: str = 'Patient_Journey_Hierarchy' # Patient Journey Hierarchy column
    brand: str = 'brand' # Brand column (medication without strength)
        
    # Possible status values
    pending: str = 'PENDING' # Pending status 'PENDING'
    active: str = 'ACTIVE' # Active status 'ACTIVE'
    cancelled: str = 'CANCELLED' # Cancelled status 'CANCELLED'
    discontinued: str = 'DISCONTINUED' # Discontinued status 'DISCONTINUED'
        
    # Possible substatus values
    pending_new: str = 'NEW' # New substatus when status is 'PENDING'
    active_shipped: str = 'SHIPMENT'# Shipment substatus when status is 'ACTIVE'
    
    # Possible PJH values
    bvpa: str = 'BV/PA' # BV/PA pjh
    intake: str = 'Intake' # Intake pjh
    fulfillment: str = 'Fulfillment' # Fulfillment pjh
    transfer: str = 'Transferred' # Transferred pjh
    payer: str = 'Payer' # Payer pjh
    no_clarity: str = 'NO STATUS CLARITY' # Final result of enrichment
        
    # Pharmacy this transform applies to
    accredo: str = 'ACCREDO'

transform = Transform()

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

# Vars
patient = transform.patient
pharm = transform.pharm
product = transform.product
ref_date = transform.ref_date
status_date = transform.status_date
status = transform.status
substatus = transform.substatus
ic_status = transform.ic_status
ic_substatus = transform.ic_substatus
pjh = transform.pjh
brand = transform.brand

# Values
pending = transform.pending
active = transform.active
cancelled = transform.cancelled
discontinued = transform.discontinued
pending_new = transform.pending_new
active_shipped = transform.active_shipped
bvpa = transform.bvpa
intake = transform.intake
fulfillment = transform.fulfillment
transfer = transform.transfer
payer = transform.payer
no_clarity = transform.no_clarity
accredo = transform.accredo

### Description

![what my transform does](assets/ds301_accredo_enrichment.png)

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[3]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

### Copy ingested data

In [ ]:
df = final_dataframe.copy()

### Impute PJH and Preprocess Data

In [ ]:
"""
This cell brings in the Patient Journey Hierarchy information from a local file.
This is just a placeholder to ensure data processes work as expected until the
true ingest data can be used.
"""

import os

pd.options.display.max_columns=999

os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config = (
    status_config
    .assign(**{
        status: lambda x:(
            x['statusCode'].str.upper()
        ),
        substatus: lambda x:(
            x['subStatus'].str.upper()
        ),
        ic_status: lambda x:(
            x['integrichain_status'].str.upper()
        ),
        ic_substatus: lambda x:(
            x['integrichain_sub_status'].str.upper()
        ),
        pjh: lambda x:(
            x['Patient_Journey_Hierarchy'].str.upper()
        )
    })
    .drop(columns=[
        'statusCode',
        'subStatus'
    ])
)

df = (
    df
    .loc[:,
        [ref_date,
         patient,
         pharm,
         product,
         status_date,
         status,
         substatus]
        ]
    .fillna('NONE')
    .assign(**{
        brand: lambda x:(
            x[product].apply(lambda x: x.split()[0].strip())
        ),
        status: lambda x:(
            x[status].str.upper()
        ),
        substatus: lambda x:(
            x[substatus].str.upper()
        )
    })
    .assign(**{
        status_date: lambda x:(
            pd.to_datetime(
                x[status_date].str[:8].astype(str),
                errors='coerce'
        )),
        ref_date: lambda x:(
            pd.to_datetime(
                x[ref_date].str[:8].astype(str),
                errors='coerce'
        )),
        'min_status_date': lambda x:(
            x.groupby([patient, pharm, brand])[status_date].transform(min)
        )
    })
    .fillna(value={ref_date: 'min_status_date'})
    .drop(columns=['min_status_date'])
    .merge(status_config, how='left', on=[status, substatus])
    .drop_duplicates()
    .sort_values(
        [patient, pharm, brand, status_date]
    )
)

### Transformation

In [ ]:
# Sort and reset index to join on later
df = (
    df
    .sort_values([patient, pharm, brand, status_date, ref_date, status])
    .reset_index(drop=False)
)

In [ ]:
# Create secondary df to find the first active shipment date for
# each patient journey
min_shipped_df = (
    df
    .loc[
        (df[ic_status] == active) &
        (df[ic_substatus] == active_shipped)
    ]
    .groupby([patient, pharm, brand])
    [status_date]
    .min()
    .reset_index()
    .rename(columns={status_date: 'first_shipped_date'})
)

In [ ]:
# Join to main df to get first active shipment date for every
# patient that has one
df = (
    df
    .merge(
        min_shipped_df,
        how='left'
    )
)

In [ ]:
# Create a column that can be grouped on later to determine the
# first date where a desired care occurs
df = (
    df
    .assign(status_spree=(
    ~(
        # Check to make sure patient journey for row above is the same
        (df[patient].eq(df[patient].shift(1))) &
        (df[pharm].eq(df[pharm].shift(1))) &
        (df[brand].eq(df[brand].shift(1))) &
        # Check to make sure statuses in row above are the same
        (df[ic_status].eq(df[ic_status].shift(1))) &
        (df[ic_substatus].eq(df[ic_substatus].shift(1)))
    )
    ).cumsum())
)

In [ ]:
# Create column denoting the min status date of each spree
df = (
    df
    .assign(min_status_date=lambda x:(
        x.groupby('status_spree')[status_date].transform(min)
    ))
)

In [ ]:
# Create bool column to denote row being right above active shipment row
df = (
    df
    .assign(above_shipment_step=lambda x:(
        # Check to make sure patient journey in row below is the same
        (x[patient].eq(x[patient].shift(-1))) &
        (x[pharm].eq(x[pharm].shift(-1))) &
        (x[brand].eq(x[brand].shift(-1))) &
        # Check to make sure status in row below is active/shipped
        (x[ic_status].shift(-1) == active) &
        (x[ic_substatus].shift(-1) == active_shipped)
    ))
)

# Pick out sprees that are above a shipment
above_shipment_series = df.groupby('status_spree').above_shipment_step.any()

# Assign True to those sprees
df = (
    df
    .assign(above_shipment=lambda x:(
        x.status_spree.isin(above_shipment_series[above_shipment_series].index)
    ))
)


In [ ]:
# Denote how many days between min_status_date and first_shipped_date
df = (
    df
    .assign(day_diff=(df['min_status_date'] - df['first_shipped_date']))
)

In [ ]:
df.loc[
    (df[patient] == accredo)
    # Negative for day diff, because we only want status sprees that
    # occur BEFORE the first active shipment
    (df.day_diff < np.timedelta64(-2, 'D')) &
    (df.above_shipment == True) &
    (
        (df[ic_status].isin([cancelled, discontinued])) |
        (
            (df[ic_status] == pending) &
            (df[ic_substatus] == pending_new)
        )
    ),
    # Change pjh to 'NO STATUS CLARITY' for the records that pass
    pjh
] = no_clarity

### Format

In [ ]:
final_dataframe = (
    df
    .set_index('index')
    .sort_index()
    .drop(columns=['first_shipped_date', 'status_spree', 'min_status_date', 'above_shipment_step', 'above_shipment', 'day_diff'])
)

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()